In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import numpy as np
import matplotlib.pyplot as plt
import math
import cv2
import random

In [ ]:
plt.figure(figsize=(28,4))
rgb_lena_img = cv2.imread('/content/drive/MyDrive/Colab Notebooks/cv_2024_1/lena.bmp', cv2.IMREAD_COLOR)
rgb_lena_img = cv2.cvtColor(rgb_lena_img, cv2.COLOR_BGR2RGB)
plt.subplot(1,7,1); plt.imshow(rgb_lena_img); plt.title('rgb_lena_img')

rgb_pepper = cv2.imread('/content/drive/MyDrive/Colab Notebooks/Image_Processing_2025_1/Image/pepper.bmp', cv2.IMREAD_COLOR)
rgb_pepper = cv2.cvtColor(rgb_pepper, cv2.COLOR_BGR2RGB)
plt.subplot(1,7,2); plt.imshow(rgb_pepper); plt.title('rgb_pepper')

rgb_landscape_forest_path = cv2.imread('/content/drive/MyDrive/Colab Notebooks/Image_Processing_2025_1/Image/Landscape-Forest Path.png', cv2.IMREAD_COLOR)
rgb_landscape_forest_path = cv2.cvtColor(rgb_landscape_forest_path, cv2.COLOR_BGR2RGB)
plt.subplot(1,7,3); plt.imshow(rgb_landscape_forest_path); plt.title('rgb_landscape_forest_path')

rgb_landscape_rocky_stream = cv2.imread('/content/drive/MyDrive/Colab Notebooks/Image_Processing_2025_1/Image/Landscape-Rocky Stream.png',cv2.IMREAD_COLOR)
rgb_landscape_rocky_stream = cv2.cvtColor(rgb_landscape_rocky_stream, cv2.COLOR_BGR2RGB)
plt.subplot(1,7,4); plt.imshow(rgb_landscape_rocky_stream); plt.title('rgb_landscape_rocky_stream')

rgb_landscape_snowy_mountains = cv2.imread('/content/drive/MyDrive/Colab Notebooks/Image_Processing_2025_1/Image/Landscape-Snowy Mountains.png',cv2.IMREAD_COLOR)
rgb_landscape_snowy_mountains = cv2.cvtColor(rgb_landscape_snowy_mountains, cv2.COLOR_BGR2RGB)
plt.subplot(1,7,5); plt.imshow(rgb_landscape_snowy_mountains); plt.title('rgb_landscape_snowy_mountains')

rgb_landscape_springtime_path = cv2.imread('/content/drive/MyDrive/Colab Notebooks/Image_Processing_2025_1/Image/Landscape_Springtime Path.png',cv2.IMREAD_COLOR)
rgb_landscape_springtime_path = cv2.cvtColor(rgb_landscape_springtime_path, cv2.COLOR_BGR2RGB)
plt.subplot(1,7,6); plt.imshow(rgb_landscape_springtime_path); plt.title('rgb_landscape_springtime_path')

rgb_landscape_sunriese_lake = cv2.imread('/content/drive/MyDrive/Colab Notebooks/Image_Processing_2025_1/Image/Landscape_Sunrise Lake.png',cv2.IMREAD_COLOR)
rgb_landscape_sunriese_lake = cv2.cvtColor(rgb_landscape_sunriese_lake, cv2.COLOR_BGR2RGB)
plt.subplot(1,7,7); plt.imshow(rgb_landscape_sunriese_lake); plt.title('rgb_landscape_sunriese_lake')


In [ ]:
# 공통으로 사용하는 함수
# PSNR 계산 함수-(height, width, channels) 형태의 RGB 이미지
def psnr_func(image1, image2):
    mse = np.mean((image1 - image2) ** 2)  # 모든 채널에 대해 MSE 계산
    psnr = 20 * math.log10(255 / math.sqrt(mse))  # PSNR 계산
    return psnr

# 가우시안 필터 생성 함수
def gaussian_filter(k, sig):
    ax = np.linspace(-(k // 2), k // 2, k)
    x, y = np.meshgrid(ax, ax)
    kernel = np.exp(-(x ** 2 + y ** 2) / (2 * sig ** 2))  # k x k 크기의 커널
    return kernel / np.sum(kernel)  # 정규화

# 가우시안 필터 마스크
gaussian_filter_mask = gaussian_filter(5, 1)

# 컨볼루션 함수
def convolution(image, filter):
    if len(image.shape) == 2:  # 흑백 이미지
        image_height, image_width = image.shape
        filtered_image = np.zeros_like(image)
    else:  # RGB 이미지
        image_height, image_width, channels = image.shape
        filtered_image = np.zeros_like(image)

    filter_height, filter_width = filter.shape
    padding = filter_height // 2

    # RGB 이미지의 경우 각 채널에 대해 컨볼루션 수행
    for y in range(padding, image_height - padding):
        for x in range(padding, image_width - padding):
            if len(image.shape) == 2:  # 흑백 이미지
                image_patch = image[y - padding:y + padding + 1, x - padding:x + padding + 1]
                filtered_image[y, x] = np.sum(image_patch * filter)
            else:  # RGB 이미지
                for c in range(channels):
                    image_patch = image[y - padding:y + padding + 1, x - padding:x + padding + 1, c]
                    filtered_image[y, x, c] = np.sum(image_patch * filter)

    return np.clip(filtered_image, 0, 255).astype(np.uint8)

# 가우시안 피라미드 생성 함수
def gaussian_pyramid(image, scale_num, levels):
    pyramid_img = [image]
    downsample_num = int(1 // scale_num)

    for _ in range(levels - 1):
        blur_img = convolution(pyramid_img[-1], gaussian_filter_mask)  # 각 채널에 대해 블러링
        # 다운샘플링
        if len(image.shape) == 2:  # 흑백 이미지
            downsample_img = blur_img[::downsample_num, ::downsample_num]
        else:  # RGB 이미지--차원수 다름
            downsample_img = blur_img[::downsample_num, ::downsample_num, :]
        pyramid_img.append(downsample_img)

    return pyramid_img  # 다운샘플된 이미지 리스트 반환

In [ ]:
def NN_interpolation(img, target_shape):
    if len(img.shape) == 2:  # 흑백 이미지
        h, w = img.shape
        new_h, new_w = target_shape
        output = np.zeros(target_shape, dtype=img.dtype)
    else:  # RGB 이미지
        h, w, channels = img.shape
        new_h, new_w = target_shape
        output = np.zeros((new_h, new_w, channels), dtype=img.dtype)

    # 출력 이미지의 크기에 대한 입력 이미지의 비율
    scale_x = w / new_w
    scale_y = h / new_h

    for i in range(new_h):  # i: y축
        for j in range(new_w):  # j: x축
            # 가장 가까운 픽셀의 좌표 계산
            x = min(int(j * scale_x), w - 1)  # 경계 처리
            y = min(int(i * scale_y), h - 1)
            if len(img.shape) == 2:  # 흑백 이미지
                output[i, j] = img[y, x]
            else:  # RGB 이미지
                output[i, j, :] = img[y, x, :]  # 모든 채널 복사

    return np.clip(output, 0, 255).astype(np.uint8)

# Bilinear interpolation (흑백 & 컬러)
def bilinear_interpolation(img, target_shape):
    if len(img.shape) == 3:  # 컬러 이미지일 경우
        h, w, c = img.shape
        new_h, new_w = target_shape
        output = np.zeros((new_h, new_w, c), dtype=img.dtype)
        for channel in range(c):
            scale_x = w / new_w
            scale_y = h / new_h
            for i in range(new_h):
                for j in range(new_w):
                    x = j * scale_x
                    y = i * scale_y
                    x0 = int(x)
                    y0 = int(y)
                    x1 = min(x0 + 1, w - 1)
                    y1 = min(y0 + 1, h - 1)
                    dx = x - x0
                    dy = y - y0
                    output[i, j, channel] = (1 - dx) * (1 - dy) * img[y0, x0, channel] + \
                                            dx * (1 - dy) * img[y0, x1, channel] + \
                                            (1 - dx) * dy * img[y1, x0, channel] + \
                                            dx * dy * img[y1, x1, channel]
    else:  # 흑백 이미지일 경우
        h, w = img.shape
        new_h, new_w = target_shape
        output = np.zeros(target_shape, dtype=img.dtype)
        scale_x = w / new_w
        scale_y = h / new_h
        for i in range(new_h):
            for j in range(new_w):
                x = j * scale_x
                y = i * scale_y
                x0 = int(x)
                y0 = int(y)
                x1 = min(x0 + 1, w - 1)
                y1 = min(y0 + 1, h - 1)
                dx = x - x0
                dy = y - y0
                output[i, j] = (1 - dx) * (1 - dy) * img[y0, x0] + \
                               dx * (1 - dy) * img[y0, x1] + \
                               (1 - dx) * dy * img[y1, x0] + \
                               dx * dy * img[y1, x1]
    return np.clip(output, 0, 255).astype(np.uint8)

# Bicubic interpolation (컬러 & 흑백)
def bicubic_interpolation(img, target_shape):
    if len(img.shape) == 3:  # 컬러 이미지일 경우
        h, w, c = img.shape
        new_h, new_w = target_shape
        output = np.zeros((new_h, new_w, c), dtype=img.dtype)
        for channel in range(c):
            scale_x = w / new_w
            scale_y = h / new_h
            for i in range(new_h):
                for j in range(new_w):
                    x = j * scale_x
                    y = i * scale_y
                    x0 = int(x)
                    y0 = int(y)
                    # Bicubic interpolation을 위해 주변 4x4 픽셀 사용
                    x_indices = [max(x0 - 1, 0), x0, min(x0 + 1, w - 1), min(x0 + 2, w - 1)]
                    y_indices = [max(y0 - 1, 0), y0, min(y0 + 1, h - 1), min(y0 + 2, h - 1)]
                    dx = x - x0
                    dy = y - y0
                    # Bicubic 커널 계산
                    def cubic(t):
                        t = np.abs(t)
                        if t <= 1:
                            return 1 - 2*t**2 + t**3
                        elif t <= 2:
                            return -t**3 + 5*t**2 - 8*t + 4
                        return 0
                    value = 0
                    for m in range(4):
                        for n in range(4):
                            weight = cubic(dx - (m - 1)) * cubic(dy - (n - 1))
                            value += weight * img[y_indices[n], x_indices[m], channel]
                    output[i, j, channel] = np.clip(value, 0, 255)
    else:  # 흑백 이미지일 경우
        h, w = img.shape
        new_h, new_w = target_shape
        output = np.zeros(target_shape, dtype=img.dtype)
        scale_x = w / new_w
        scale_y = h / new_h
        for i in range(new_h):
            for j in range(new_w):
                x = j * scale_x
                y = i * scale_y
                x0 = int(x)
                y0 = int(y)
                x_indices = [max(x0 - 1, 0), x0, min(x0 + 1, w - 1), min(x0 + 2, w - 1)]
                y_indices = [max(y0 - 1, 0), y0, min(y0 + 1, h - 1), min(y0 + 2, h - 1)]
                dx = x - x0
                dy = y - y0
                def cubic(t):
                    t = np.abs(t)
                    if t <= 1:
                        return 1 - 2*t**2 + t**3
                    elif t <= 2:
                        return -t**3 + 5*t**2 - 8*t + 4
                    return 0
                value = 0
                for m in range(4):
                    for n in range(4):
                        weight = cubic(dx - (m - 1)) * cubic(dy - (n - 1))
                        value += weight * img[y_indices[n], x_indices[m]]
                output[i, j] = np.clip(value, 0, 255)
    return np.clip(output, 0, 255).astype(np.uint8)

def b_spline_interpolation(img, target_shape):
    # 3차 B-spline 커널 함수: 거리 u에 대한 가중치 계산
    def bspline_kernel(u):
        u = abs(u)
        if u < 1:
            return (1/6) * (3 * u**3 - 6 * u**2 + 4)
        elif u < 2:
            return (1/6) * (-u**3 + 6 * u**2 - 12 * u + 8)
        return 0

    if len(img.shape) == 2:  # 흑백 이미지
        h, w = img.shape
        new_h, new_w = target_shape
        output = np.zeros(target_shape, dtype=img.dtype)
    else:  # RGB 이미지
        h, w, channels = img.shape
        new_h, new_w = target_shape
        output = np.zeros((new_h, new_w, channels), dtype=img.dtype)

    scale_x = w / new_w
    scale_y = h / new_h

    for i in range(new_h):
        for j in range(new_w):
            # 출력 픽셀 (i, j)에 대응하는 입력 이미지의 좌표
            x = j * scale_x
            y = i * scale_y

            # 정수 좌표와 소수 부분 계산
            x_int = int(x)
            y_int = int(y)
            x_frac = x - x_int
            y_frac = y - y_int

            if len(img.shape) == 2:  # 흑백 이미지
                # 4x4 주변 픽셀에 대해 B-spline 가중치 계산
                value = 0.0
                for m in range(-1, 3):  # y 방향: -1, 0, 1, 2
                    for n in range(-1, 3):  # x 방향: -1, 0, 1, 2
                        # 입력 이미지의 좌표
                        x_idx = min(max(x_int + n, 0), w - 1)  # 경계 처리
                        y_idx = min(max(y_int + m, 0), h - 1)  # 경계 처리

                        # B-spline 커널 가중치 계산
                        weight_x = bspline_kernel(n - x_frac)
                        weight_y = bspline_kernel(m - y_frac)
                        weight = weight_x * weight_y

                        # 가중치 적용하여 픽셀 값 누적
                        value += img[y_idx, x_idx] * weight

                # 출력 픽셀 값 저장
                output[i, j] = np.clip(round(value), 0, 255).astype(img.dtype)
            else:  # RGB 이미지
                # 각 채널에 대해 B-spline 보간 수행
                for c in range(channels):
                    value = 0.0
                    for m in range(-1, 3):  # y 방향
                        for n in range(-1, 3):  # x 방향
                            x_idx = min(max(x_int + n, 0), w - 1)
                            y_idx = min(max(y_int + m, 0), h - 1)

                            weight_x = bspline_kernel(n - x_frac)
                            weight_y = bspline_kernel(m - y_frac)
                            weight = weight_x * weight_y

                            value += img[y_idx, x_idx, c] * weight

                    output[i, j, c] = np.clip(round(value), 0, 255).astype(img.dtype)

    return np.clip(output, 0, 255).astype(np.uint8)

In [ ]:
# 업샘플링 이후 가우시안 스무딩 적용
def upsample(img, target_shape):
    # 입력 이미지와 목표 크기의 비율 계산
    scale_x = target_shape[0] // img.shape[0]
    scale_y = target_shape[1] // img.shape[1]
    interpolated_img = NN_interpolation(img, (scale_x * img.shape[0], scale_y * img.shape[1]))
    # 계단 현상 완화를 위한 가우시안 필터링 적용
    final_upsampled_img = convolution(interpolated_img, gaussian_filter_mask)
    return final_upsampled_img

# 3가지 해상도의 이미지를 결합하여 최종 이미지(512x512) 구성
def reconstruct_image(img_128, img_256, img_512):
    up_128 = upsample(img_128, (512, 512))  # 128 --> 512 업샘플링
    up_256 = upsample(img_256, (512, 512))  # 256 --> 512 업샘플링
    # 업샘플링된 이미지를 가중치를 적용해서 하나로 결합
    reconstructed_img = 0.4 * up_128 + 0.3 * up_256 + 0.3 * img_512
    return np.clip(reconstructed_img, 0, 255)  # 픽셀값 0~255로 제한


In [ ]:
# RGB 이미지:분산 기반 영역 추출
def variance_based_selection(img, k):
    patch_size = k
    num_patches = 16384 // (patch_size**2)

    if len(img.shape) == 2:  # 흑백 이미지
        h, w = img.shape
        output = np.zeros_like(img)
    else:  # RGB 이미지
        h, w, channels = img.shape
        output = np.zeros_like(img)

    variances = []

    # 모든 가능한 패치의 분산 계산
    for i in range(0, h - patch_size, patch_size):
        for j in range(0, w - patch_size, patch_size):
            patch = img[i:i+patch_size, j:j+patch_size]
            if len(img.shape) == 2:  # 흑백 이미지
                var = np.var(patch)  # 단일 채널 분산
            else:  # RGB 이미지
                # 각 채널의 분산을 계산하고 평균
                var = np.mean([np.var(patch[:, :, c]) for c in range(channels)])
            variances.append((var, i, j))  # 분산과 패치 좌표 저장

    # 분산이 높은 순서로 정렬
    variances.sort(reverse=True)
    selected_patches = variances[:num_patches]  # 상위 num_patches 선택

    # 선택된 패치를 출력 이미지에 복사
    for _, i, j in selected_patches:
        output[i:i+patch_size, j:j+patch_size] = img[i:i+patch_size, j:j+patch_size]

    return output

gaus_pyramid_img = gaussian_pyramid(rgb_lena_img, 0.5, 3)  # 0.5비율로 scaling, 3계층
img_512 = gaus_pyramid_img[0]  # 512x512x3
img_256 = gaus_pyramid_img[1]  # 256x256x3
img_128 = gaus_pyramid_img[2]  # 128x128x3

# k=16으로 분산기 패치 추출
img_256_sparse = variance_based_selection(img_256, 16)
img_512_sparse = variance_based_selection(img_512, 16)

# 이미지 재구성
img_reconstructed = reconstruct_image(img_128, img_256_sparse, img_512_sparse)


In [ ]:
# psnr 결과
score = psnr_func(rgb_lena_img, img_reconstructed)
print(f"PSNR of reconstructed image: {score:.4f}")

plt.figure(figsize=(15,5))
plt.subplot(1,3,1); plt.imshow(img_128); plt.title('img_128')
plt.subplot(1,3,2); plt.imshow(img_256_sparse); plt.title('img_256_sparse')
plt.subplot(1,3,3); plt.imshow(img_512_sparse); plt.title('img_512_sparse')
plt.show()
# Optional img vs Reconstructed img
img_reconstructed = img_reconstructed.astype(np.uint8)
plt.figure(figsize=(10,5))
plt.subplot(1,2,1); plt.title("Original img"); plt.imshow(rgb_lena_img)
plt.subplot(1,2,2); plt.title("Reconstructed img"); plt.imshow(img_reconstructed)
plt.show()


In [ ]:
# k=2일 때 분산기반 패치 추출(k=1일 경우 너무 작은 값으로 인해 분산 계산 진행 오류 발생)
img_256_sparse = variance_based_selection(img_256, 2)
img_512_sparse = variance_based_selection(img_512, 2)

# 이미지 재구성
img_reconstructed = reconstruct_image(img_128, img_256_sparse, img_512_sparse)

# psnr 결과
score = psnr_func(rgb_lena_img, img_reconstructed)
print(f"PSNR of reconstructed image: {score:.4f}")

plt.figure(figsize=(15,5))
plt.subplot(1,3,1); plt.imshow(img_128); plt.title('img_128')
plt.subplot(1,3,2); plt.imshow(img_256_sparse); plt.title('img_256_sparse')
plt.subplot(1,3,3); plt.imshow(img_512_sparse); plt.title('img_512_sparse')
plt.show()
# Optional img vs Reconstructed img
img_reconstructed = img_reconstructed.astype(np.uint8)
plt.figure(figsize=(10,5))
plt.subplot(1,2,1); plt.title("Original img"); plt.imshow(rgb_lena_img)
plt.subplot(1,2,2); plt.title("Reconstructed img"); plt.imshow(img_reconstructed)
plt.show()


In [ ]:
# k=4일 때 분산기반 패치 추출
img_256_sparse = variance_based_selection(img_256, 4)
img_512_sparse = variance_based_selection(img_512, 4)

# 이미지 재구성
img_reconstructed = reconstruct_image(img_128, img_256_sparse, img_512_sparse)

# psnr 결과
score = psnr_func(rgb_lena_img, img_reconstructed)
print(f"PSNR of reconstructed image: {score:.4f}")

plt.figure(figsize=(15,5))
plt.subplot(1,3,1); plt.imshow(img_128); plt.title('img_128')
plt.subplot(1,3,2); plt.imshow(img_256_sparse); plt.title('img_256_sparse')
plt.subplot(1,3,3); plt.imshow(img_512_sparse); plt.title('img_512_sparse')
plt.show()
# Optional img vs Reconstructed img
img_reconstructed = img_reconstructed.astype(np.uint8)
plt.figure(figsize=(10,5))
plt.subplot(1,2,1); plt.title("Original img"); plt.imshow(rgb_lena_img)
plt.subplot(1,2,2); plt.title("Reconstructed img"); plt.imshow(img_reconstructed)
plt.show()


In [ ]:
# k=8일 때 분산기반 패치 추출
img_256_sparse = variance_based_selection(img_256, 8)
img_512_sparse = variance_based_selection(img_512, 8)

# 이미지 재구성
img_reconstructed = reconstruct_image(img_128, img_256_sparse, img_512_sparse)

# psnr 결과
score = psnr_func(rgb_lena_img, img_reconstructed)
print(f"PSNR of reconstructed image: {score:.4f}")

plt.figure(figsize=(15,5))
plt.subplot(1,3,1); plt.imshow(img_128); plt.title('img_128')
plt.subplot(1,3,2); plt.imshow(img_256_sparse); plt.title('img_256_sparse')
plt.subplot(1,3,3); plt.imshow(img_512_sparse); plt.title('img_512_sparse')
plt.show()
# Optional img vs Reconstructed img
img_reconstructed = img_reconstructed.astype(np.uint8)
plt.figure(figsize=(10,5))
plt.subplot(1,2,1); plt.title("Original img"); plt.imshow(rgb_lena_img)
plt.subplot(1,2,2); plt.title("Reconstructed img"); plt.imshow(img_reconstructed)
plt.show()


In [ ]:
# k=32일 때 분산기반 패치 추출
img_256_sparse = variance_based_selection(img_256, 32)
img_512_sparse = variance_based_selection(img_512, 32)

# 이미지 재구성
img_reconstructed = reconstruct_image(img_128, img_256_sparse, img_512_sparse)

# psnr 결과
score = psnr_func(rgb_lena_img, img_reconstructed)
print(f"PSNR of reconstructed image: {score:.4f}")

plt.figure(figsize=(15,5))
plt.subplot(1,3,1); plt.imshow(img_128); plt.title('img_128')
plt.subplot(1,3,2); plt.imshow(img_256_sparse); plt.title('img_256_sparse')
plt.subplot(1,3,3); plt.imshow(img_512_sparse); plt.title('img_512_sparse')
plt.show()
# Optional img vs Reconstructed img
img_reconstructed = img_reconstructed.astype(np.uint8)
plt.figure(figsize=(10,5))
plt.subplot(1,2,1); plt.title("Original img"); plt.imshow(rgb_lena_img)
plt.subplot(1,2,2); plt.title("Reconstructed img"); plt.imshow(img_reconstructed)
plt.show()


In [ ]:
# k=16일 때 분산기반 패치 추출
img_256_sparse = gradient_based_selection(img_256, 16)
img_512_sparse = gradient_based_selection(img_512, 16)

# 이미지 재구성
img_reconstructed = reconstruct_image(img_128, img_256_sparse, img_512_sparse)


In [ ]:
# psnr 결과
score = psnr_func(rgb_lena_img, img_reconstructed)
print(f"PSNR of reconstructed image: {score:.4f}")

plt.figure(figsize=(15,5))
plt.subplot(1,3,1); plt.imshow(img_128); plt.title('img_128')
plt.subplot(1,3,2); plt.imshow(img_256_sparse); plt.title('img_256_sparse')
plt.subplot(1,3,3); plt.imshow(img_512_sparse); plt.title('img_512_sparse')
plt.show()
# Optional img vs Reconstructed img
img_reconstructed = img_reconstructed.astype(np.uint8)
plt.figure(figsize=(10,5))
plt.subplot(1,2,1); plt.title("Original img"); plt.imshow(rgb_lena_img)
plt.subplot(1,2,2); plt.title("Reconstructed img"); plt.imshow(img_reconstructed)
plt.show()


In [ ]:
# k=1으로 랜덤 패치 추출
img_256_sparse = gradient_based_selection(img_256, 1)
img_512_sparse = gradient_based_selection(img_512, 1)

# 이미지 재구성
img_reconstructed = reconstruct_image(img_128, img_256_sparse, img_512_sparse)

# psnr 결과
score = psnr_func(rgb_lena_img, img_reconstructed)
print(f"PSNR of reconstructed image: {score:.4f}")

plt.figure(figsize=(15,5))
plt.subplot(1,3,1); plt.imshow(img_128); plt.title('img_128')
plt.subplot(1,3,2); plt.imshow(img_256_sparse); plt.title('img_256_sparse')
plt.subplot(1,3,3); plt.imshow(img_512_sparse); plt.title('img_512_sparse')
plt.show()
# Optional img vs Reconstructed img
img_reconstructed = img_reconstructed.astype(np.uint8)
plt.figure(figsize=(10,5))
plt.subplot(1,2,1); plt.title("Original img"); plt.imshow(rgb_lena_img)
plt.subplot(1,2,2); plt.title("Reconstructed img"); plt.imshow(img_reconstructed)
plt.show()


In [ ]:
# k=2으로 랜덤 패치 추출
img_256_sparse = gradient_based_selection(img_256, 2)
img_512_sparse = gradient_based_selection(img_512, 2)

# 이미지 재구성
img_reconstructed = reconstruct_image(img_128, img_256_sparse, img_512_sparse)

# psnr 결과
score = psnr_func(rgb_lena_img, img_reconstructed)
print(f"PSNR of reconstructed image: {score:.4f}")

plt.figure(figsize=(15,5))
plt.subplot(1,3,1); plt.imshow(img_128); plt.title('img_128')
plt.subplot(1,3,2); plt.imshow(img_256_sparse); plt.title('img_256_sparse')
plt.subplot(1,3,3); plt.imshow(img_512_sparse); plt.title('img_512_sparse')
plt.show()
# Optional img vs Reconstructed img
img_reconstructed = img_reconstructed.astype(np.uint8)
plt.figure(figsize=(10,5))
plt.subplot(1,2,1); plt.title("Original img"); plt.imshow(rgb_lena_img)
plt.subplot(1,2,2); plt.title("Reconstructed img"); plt.imshow(img_reconstructed)
plt.show()


In [ ]:
# k=4으로 랜덤 패치 추출
img_256_sparse = gradient_based_selection(img_256, 4)
img_512_sparse = gradient_based_selection(img_512, 4)

# 이미지 재구성
img_reconstructed = reconstruct_image(img_128, img_256_sparse, img_512_sparse)

# psnr 결과
score = psnr_func(rgb_lena_img, img_reconstructed)
print(f"PSNR of reconstructed image: {score:.4f}")

plt.figure(figsize=(15,5))
plt.subplot(1,3,1); plt.imshow(img_128); plt.title('img_128')
plt.subplot(1,3,2); plt.imshow(img_256_sparse); plt.title('img_256_sparse')
plt.subplot(1,3,3); plt.imshow(img_512_sparse); plt.title('img_512_sparse')
plt.show()
# Optional img vs Reconstructed img
img_reconstructed = img_reconstructed.astype(np.uint8)
plt.figure(figsize=(10,5))
plt.subplot(1,2,1); plt.title("Original img"); plt.imshow(rgb_lena_img)
plt.subplot(1,2,2); plt.title("Reconstructed img"); plt.imshow(img_reconstructed)
plt.show()


In [ ]:
# k=8으로 랜덤 패치 추출
img_256_sparse = gradient_based_selection(img_256, 8)
img_512_sparse = gradient_based_selection(img_512, 8)

# 이미지 재구성
img_reconstructed = reconstruct_image(img_128, img_256_sparse, img_512_sparse)

# psnr 결과
score = psnr_func(rgb_lena_img, img_reconstructed)
print(f"PSNR of reconstructed image: {score:.4f}")

plt.figure(figsize=(15,5))
plt.subplot(1,3,1); plt.imshow(img_128); plt.title('img_128')
plt.subplot(1,3,2); plt.imshow(img_256_sparse); plt.title('img_256_sparse')
plt.subplot(1,3,3); plt.imshow(img_512_sparse); plt.title('img_512_sparse')
plt.show()
# Optional img vs Reconstructed img
img_reconstructed = img_reconstructed.astype(np.uint8)
plt.figure(figsize=(10,5))
plt.subplot(1,2,1); plt.title("Original img"); plt.imshow(rgb_lena_img)
plt.subplot(1,2,2); plt.title("Reconstructed img"); plt.imshow(img_reconstructed)
plt.show()


In [ ]:
# k=32으로 랜덤 패치 추출
img_256_sparse = gradient_based_selection(img_256, 32)
img_512_sparse = gradient_based_selection(img_512, 32)

# 이미지 재구성
img_reconstructed = reconstruct_image(img_128, img_256_sparse, img_512_sparse)

# psnr 결과
score = psnr_func(rgb_lena_img, img_reconstructed)
print(f"PSNR of reconstructed image: {score:.4f}")

plt.figure(figsize=(15,5))
plt.subplot(1,3,1); plt.imshow(img_128); plt.title('img_128')
plt.subplot(1,3,2); plt.imshow(img_256_sparse); plt.title('img_256_sparse')
plt.subplot(1,3,3); plt.imshow(img_512_sparse); plt.title('img_512_sparse')
plt.show()
# Optional img vs Reconstructed img
img_reconstructed = img_reconstructed.astype(np.uint8)
plt.figure(figsize=(10,5))
plt.subplot(1,2,1); plt.title("Original img"); plt.imshow(rgb_lena_img)
plt.subplot(1,2,2); plt.title("Reconstructed img"); plt.imshow(img_reconstructed)
plt.show()


In [ ]:
#다른 사진들로 진행
# 1번 사진
gaus_pyramid_img = gaussian_pyramid(rgb_pepper, 0.5, 3)  # 0.5비율로 scaling, 3계층
img_512 = gaus_pyramid_img[0]  # 512x512x3
img_256 = gaus_pyramid_img[1]  # 256x256x3
img_128 = gaus_pyramid_img[2]  # 128x128x3

# k=16으로 랜덤 패치 추출
img_256_sparse = variance_based_selection(img_256, 2)
img_512_sparse = variance_based_selection(img_512, 2)

# 이미지 재구성
img_reconstructed = reconstruct_image(img_128, img_256_sparse, img_512_sparse)


In [ ]:
# psnr 결과
score = psnr_func(rgb_pepper, img_reconstructed)
print(f"PSNR of reconstructed image: {score:.4f}")

plt.figure(figsize=(15,5))
plt.subplot(1,3,1); plt.imshow(img_128); plt.title('img_128')
plt.subplot(1,3,2); plt.imshow(img_256_sparse); plt.title('img_256_sparse')
plt.subplot(1,3,3); plt.imshow(img_512_sparse); plt.title('img_512_sparse')
plt.show()
# Optional img vs Reconstructed img
img_reconstructed = img_reconstructed.astype(np.uint8)
plt.figure(figsize=(10,5))
plt.subplot(1,2,1); plt.title("Original img"); plt.imshow(rgb_pepper)
plt.subplot(1,2,2); plt.title("Reconstructed img"); plt.imshow(img_reconstructed)
plt.show()


In [ ]:
# 2번 사진
gaus_pyramid_img = gaussian_pyramid(rgb_landscape_forest_path, 0.5, 3)  # 0.5비율로 scaling, 3계층
img_512 = gaus_pyramid_img[0]  # 512x512x3
img_256 = gaus_pyramid_img[1]  # 256x256x3
img_128 = gaus_pyramid_img[2]  # 128x128x3

# k=16으로 랜덤 패치 추출
img_256_sparse = variance_based_selection(img_256, 32)
img_512_sparse = variance_based_selection(img_512, 32)

# 이미지 재구성
img_reconstructed = reconstruct_image(img_128, img_256_sparse, img_512_sparse)


In [ ]:
# psnr 결과
score = psnr_func(rgb_landscape_forest_path, img_reconstructed)
print(f"PSNR of reconstructed image: {score:.4f}")

plt.figure(figsize=(15,5))
plt.subplot(1,3,1); plt.imshow(img_128); plt.title('img_128')
plt.subplot(1,3,2); plt.imshow(img_256_sparse); plt.title('img_256_sparse')
plt.subplot(1,3,3); plt.imshow(img_512_sparse); plt.title('img_512_sparse')
plt.show()
# Optional img vs Reconstructed img
img_reconstructed = img_reconstructed.astype(np.uint8)
plt.figure(figsize=(10,5))
plt.subplot(1,2,1); plt.title("Original img"); plt.imshow(rgb_landscape_forest_path)
plt.subplot(1,2,2); plt.title("Reconstructed img"); plt.imshow(img_reconstructed)
plt.show()


In [ ]:
# 3번 사진
gaus_pyramid_img = gaussian_pyramid(rgb_landscape_rocky_stream, 0.5, 3)  # 0.5비율로 scaling, 3계층
img_512 = gaus_pyramid_img[0]  # 512x512x3
img_256 = gaus_pyramid_img[1]  # 256x256x3
img_128 = gaus_pyramid_img[2]  # 128x128x3

# k=16으로 랜덤 패치 추출
img_256_sparse = variance_based_selection(img_256, 32)
img_512_sparse = variance_based_selection(img_512, 32)

# 이미지 재구성
img_reconstructed = reconstruct_image(img_128, img_256_sparse, img_512_sparse)


In [ ]:
# psnr 결과
score = psnr_func(rgb_landscape_rocky_stream, img_reconstructed)
print(f"PSNR of reconstructed image: {score:.4f}")

plt.figure(figsize=(15,5))
plt.subplot(1,3,1); plt.imshow(img_128); plt.title('img_128')
plt.subplot(1,3,2); plt.imshow(img_256_sparse); plt.title('img_256_sparse')
plt.subplot(1,3,3); plt.imshow(img_512_sparse); plt.title('img_512_sparse')
plt.show()
# Optional img vs Reconstructed img
img_reconstructed = img_reconstructed.astype(np.uint8)
plt.figure(figsize=(10,5))
plt.subplot(1,2,1); plt.title("Original img"); plt.imshow(rgb_landscape_rocky_stream)
plt.subplot(1,2,2); plt.title("Reconstructed img"); plt.imshow(img_reconstructed)
plt.show()


In [ ]:
# 4번 사진
gaus_pyramid_img = gaussian_pyramid(rgb_landscape_snowy_mountains, 0.5, 3)  # 0.5비율로 scaling, 3계층
img_512 = gaus_pyramid_img[0]  # 512x512x3
img_256 = gaus_pyramid_img[1]  # 256x256x3
img_128 = gaus_pyramid_img[2]  # 128x128x3

# k=16으로 랜덤 패치 추출
img_256_sparse = variance_based_selection(img_256, 32)
img_512_sparse = variance_based_selection(img_512, 32)

# 이미지 재구성
img_reconstructed = reconstruct_image(img_128, img_256_sparse, img_512_sparse)


In [ ]:
# psnr 결과
score = psnr_func(rgb_landscape_snowy_mountains, img_reconstructed)
print(f"PSNR of reconstructed image: {score:.4f}")

plt.figure(figsize=(15,5))
plt.subplot(1,3,1); plt.imshow(img_128); plt.title('img_128')
plt.subplot(1,3,2); plt.imshow(img_256_sparse); plt.title('img_256_sparse')
plt.subplot(1,3,3); plt.imshow(img_512_sparse); plt.title('img_512_sparse')
plt.show()
# Optional img vs Reconstructed img
img_reconstructed = img_reconstructed.astype(np.uint8)
plt.figure(figsize=(10,5))
plt.subplot(1,2,1); plt.title("Original img"); plt.imshow(rgb_landscape_snowy_mountains)
plt.subplot(1,2,2); plt.title("Reconstructed img"); plt.imshow(img_reconstructed)
plt.show()


In [ ]:
# 5번 사진
gaus_pyramid_img = gaussian_pyramid(rgb_landscape_springtime_path, 0.5, 3)  # 0.5비율로 scaling, 3계층
img_512 = gaus_pyramid_img[0]  # 512x512x3
img_256 = gaus_pyramid_img[1]  # 256x256x3
img_128 = gaus_pyramid_img[2]  # 128x128x3

# k=16으로 랜덤 패치 추출
img_256_sparse = variance_based_selection(img_256, 32)
img_512_sparse = variance_based_selection(img_512, 32)

# 이미지 재구성
img_reconstructed = reconstruct_image(img_128, img_256_sparse, img_512_sparse)


In [ ]:
# psnr 결과
score = psnr_func(rgb_landscape_springtime_path, img_reconstructed)
print(f"PSNR of reconstructed image: {score:.4f}")

plt.figure(figsize=(15,5))
plt.subplot(1,3,1); plt.imshow(img_128); plt.title('img_128')
plt.subplot(1,3,2); plt.imshow(img_256_sparse); plt.title('img_256_sparse')
plt.subplot(1,3,3); plt.imshow(img_512_sparse); plt.title('img_512_sparse')
plt.show()
# Optional img vs Reconstructed img
img_reconstructed = img_reconstructed.astype(np.uint8)
plt.figure(figsize=(10,5))
plt.subplot(1,2,1); plt.title("Original img"); plt.imshow(rgb_landscape_springtime_path)
plt.subplot(1,2,2); plt.title("Reconstructed img"); plt.imshow(img_reconstructed)
plt.show()


In [ ]:
#6번 사진
gaus_pyramid_img = gaussian_pyramid(rgb_landscape_sunriese_lake, 0.5, 3)  # 0.5비율로 scaling, 3계층
img_512 = gaus_pyramid_img[0]  # 512x512x3
img_256 = gaus_pyramid_img[1]  # 256x256x3
img_128 = gaus_pyramid_img[2]  # 128x128x3

# k=16으로 랜덤 패치 추출
img_256_sparse = variance_based_selection(img_256, 32)
img_512_sparse = variance_based_selection(img_512, 32)

# 이미지 재구성
img_reconstructed = reconstruct_image(img_128, img_256_sparse, img_512_sparse)


In [ ]:
# psnr 결과
score = psnr_func(rgb_landscape_sunriese_lake, img_reconstructed)
print(f"PSNR of reconstructed image: {score:.4f}")

plt.figure(figsize=(15,5))
plt.subplot(1,3,1); plt.imshow(img_128); plt.title('img_128')
plt.subplot(1,3,2); plt.imshow(img_256_sparse); plt.title('img_256_sparse')
plt.subplot(1,3,3); plt.imshow(img_512_sparse); plt.title('img_512_sparse')
plt.show()
# Optional img vs Reconstructed img
img_reconstructed = img_reconstructed.astype(np.uint8)
plt.figure(figsize=(10,5))
plt.subplot(1,2,1); plt.title("Original img"); plt.imshow(rgb_landscape_sunriese_lake)
plt.subplot(1,2,2); plt.title("Reconstructed img"); plt.imshow(img_reconstructed)
plt.show()


In [ ]:
#다른 사진들로 진행=======에지기반 영역 선택
# 1번 사진
gaus_pyramid_img = gaussian_pyramid(rgb_pepper, 0.5, 3)  # 0.5비율로 scaling, 3계층
img_512 = gaus_pyramid_img[0]  # 512x512x3
img_256 = gaus_pyramid_img[1]  # 256x256x3
img_128 = gaus_pyramid_img[2]  # 128x128x3

# k=16으로 랜덤 패치 추출
img_256_sparse = gradient_based_selection(img_256, 32)
img_512_sparse = gradient_based_selection(img_512, 32)

# 이미지 재구성
img_reconstructed = reconstruct_image(img_128, img_256_sparse, img_512_sparse)
